# Lab 2 · QSAR Potency Prediction

We will build a Quantitative Structure–Activity Relationship (QSAR) model that predicts pIC₅₀ using simple molecular descriptors.

### Learning goals
- Practice loading a CSV file with small-molecule descriptors.
- Visualize how each descriptor contributes to activity.
- Fit a multi-feature linear regression model.
- Use the trained model to score a new compound idea.

### Workflow
1. Import libraries and load `../data/qsar_protease_potency.csv`.
2. Inspect and plot the data to understand descriptor ranges.
3. Train/test split, fit the model, and examine coefficients.
4. Predict pIC₅₀ for held-out compounds and a hypothetical design.

In [ ]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

data_path = Path('..') / 'data' / 'qsar_protease_potency.csv'
df = pd.read_csv(data_path)
print(f"Loaded {len(df)} compounds.")
df.head()

### Exercise 1 · Explore descriptors
Look at the summary statistics and scatter each descriptor against activity.

In [ ]:
df.describe()

fig, axes = plt.subplots(1, 3, figsize=(12, 4))
features = ['mol_weight_da', 'logP', 'polar_surface_area']
for ax, feature in zip(axes, features):
    ax.scatter(df[feature], df['activity_pIC50'], color='teal')
    ax.set_xlabel(feature)
    ax.set_ylabel('pIC50')
fig.suptitle('Descriptor trends')
plt.tight_layout()
plt.show()

### Exercise 2 · Train the regression model

In [ ]:
feature_cols = ['mol_weight_da', 'logP', 'polar_surface_area']
X = df[feature_cols]
y = df['activity_pIC50']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=7
)

model = LinearRegression()
model.fit(X_train, y_train)

print(f"Intercept: {model.intercept_:.3f}")
for name, coef in zip(feature_cols, model.coef_):
    print(f"{name}: {coef:.4f}")

### Exercise 3 · Evaluate with held-out compounds

In [ ]:
y_pred = model.predict(X_test)

r2 = r2_score(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"R²: {r2:.3f}")
print(f"RMSE: {rmse:.3f} pIC50 units")

comparison = X_test.copy()
comparison['actual_pIC50'] = y_test.values
comparison['predicted_pIC50'] = y_pred.round(2)
comparison

### Exercise 4 · Score a new compound idea
Change the numbers below to simulate a different design and re-run the cell.

In [ ]:
new_compound = pd.DataFrame(
    {
        'mol_weight_da': [320],
        'logP': [3.1],
        'polar_surface_area': [37]
    },
    index=['NewIdea']
)

predicted_activity = model.predict(new_compound)[0]
print(f"Predicted pIC50: {predicted_activity:.2f}")
new_compound['predicted_pIC50'] = predicted_activity
new_compound

### Reflection prompt
Which descriptor had the strongest coefficient? Does that match the scatter plots you observed?